# Infocard Prototype

This prototype seeks to sketch out possible interfaces that can pull data from names authority databases and supplement the Primary Source Coop.

Cheverus

Links:

SNAC ID -> SNAC Bio, Resources

LCNAF ID -> Wikidata Q ID, VIAF
https://id.loc.gov/authorities/names/n80001490.html

Wikidata -> Image Field (p18), Gender (p21), Occupation (p106), Position held (p39)

Wikidata Query Service: https://query.wikidata.org/

VIAF -> Works

## Linked Open Data

* Making documents discoverable
* Internal Coop links amplify with Wikidata, etc.?
* Reverse link (wikidata, etc. id to xml-id)
* List of Coop references

Wikidata congressmen during timeframe and find them in dJQA.

## Notes

* Messy
    * There are series of responses required to get some data: LOC -> VIAF -> VIAF.xml
    * Pro: LOC gets VIAF; Con: slows down responsivity
    
* To Do
    * Return list of all documents that reference individual persRef

In [55]:
import requests, re, warnings, pprint
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

import dash, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash

# Ignore simple warnings.
warnings.simplefilter('ignore', DeprecationWarning)

abs_dir = '/Users/quinn.wi/Documents/Data/JQA_pre-2021-04-14/'

In [2]:
%%time

names_data = pd.read_excel(abs_dir + 'DJQA_Names-List_singleSheet.xlsx', index_col = None) 

names_data.columns = names_data.columns.str.replace('\s', '_')

# names_data = names_data.query('(Last_Name == "Randolph") & (First_Name == "John")')

names_data = names_data.dropna(subset = ['LC_Name_Authority'])

CPU times: user 1min 7s, sys: 676 ms, total: 1min 8s
Wall time: 1min 9s


In [3]:
%%time

names_data.head(1)

CPU times: user 331 µs, sys: 17 µs, total: 348 µs
Wall time: 345 µs


,Last_Name,First_Name,Middle_Name,Maiden_Name,Variant_form_of_name,Title,Suffix,Short-hand_option_for_name,Hyogebated-unique-string-of-characters,Birth_Date,...,notes_for_editorial_team,Notes,Source,URL,LC_Name_Authority,SNAC,Identifier's_initials_and_date,project,Date_First_Mentioned,Second_URL
1419,Adams,Louisa,Catherine,Johnson,NaN,NaN,NaN,NaN,adams-louisa-catherine,1775,...,NaN,wife of JQA,Adams Biographical Sketches at MHS website,http://www.masshist.org/2012/adams/biographies...,n 86022545,NaN,KNB 9/8/2018,NaN,NaN,NaN


## Functions

In [137]:
%%time

def parseLOC(identifier):
#     Lookup URI.
    url = f"https://id.loc.gov/authorities/names/{identifier}.madsxml.xml"
    response = requests.get(url).text
    
#     Parse XML with namespace from string.
    root = ET.fromstring(response)
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    
#     Gather information.
    loc_xpath = {'Name': './/ns:name[@authority="naf"]/ns:namePart', 
                 'Birth & Death Date': './/ns:name[@authority="naf"]/ns:namePart[@type="date"]', 
                 'Gendered Term': './/ns:genderTerm'}

    loc_data = {}
    loc_data['id'] = identifier
    
    for k, v in loc_xpath.items():
        try:
            loc_data[k] = root.find(v, ns).text
        except:
            loc_data[k] = 'NaN'
    
    return loc_data
    

# Read LOC html to get Wikidata & VIAF.
def locSoup(identifier):
    url = f"https://id.loc.gov/authorities/names/{identifier}.html"
    response = requests.get(url).text
    locSoup = BeautifulSoup(response)

    wiki = locSoup.find('span', {'href': re.compile(r'http://www.wikidata.org/entity/.*')})['href']
    wiki_id = re.search(r'.*/(Q.*)', wiki).group(1)
    
    viaf = locSoup.find('a', {'href': re.compile(r'http://viaf.org/viaf/.*')})['href']
    
    return {'wiki': {'id': wiki_id, 'url': wiki}, 
            'viaf': {'url': viaf}}
    

def parseSNAC(identifier):
#     Lookup URI & get JSON format.
    url = f"https://snaccooperative.org/download/{identifier}?type=constellation_json"
    response = requests.get(url).json()

    namePart = response['nameEntries'][0]['original']
    birthDeathDate = re.search('\d{4}-\d{4}', namePart).group(0)
    
    return {'SNAC ID': identifier, 'Name': namePart, 'Birth & Death Date': birthDeathDate}

# Use SPARQL to gather Wikidata.
# Run this function after locSoup.
def sparqlWiki(data):
    wiki_key = data['wiki']['id']
    url = 'https://query.wikidata.org/sparql'
    
    query = f"""

    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?personLabel ?genderLabel ?birthDate ?deathDate ?occupationLabel

    WHERE 
    {{
      wd:{wiki_key} rdfs:label ?person ;
                wdt:P18 ?image ;
                wdt:P21 ?gender ;
                wdt:P569 ?birthDate ;
                wdt:P570 ?deathDate ;
                wdt:P106 ?occupationLabel .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
      FILTER ( LANG ( ?person ) = 'en' )
    }}
    """
    
    resp = requests.get(url, params = {'format': 'json', 'query': query}).json()
    
#     data['wiki']['image'] = resp['results']['bindings'][0]['image']['value']
    data['wiki']['genderLabel'] = resp['results']['bindings'][0]['genderLabel']['value']
    data['wiki']['birthDate'] = resp['results']['bindings'][0]['birthDate']['value']
    data['wiki']['deathDate'] = resp['results']['bindings'][0]['deathDate']['value']
    
    data['wiki']['occupations'] = []
    for o in resp['results']['bindings']:
        data['wiki']['occupations'].append(o['occupationLabel']['value'])
        
        
# Gather VIAF data.
# Run after locSoup.
def parseVIAF(data):
    url = data['viaf']['url']
    response = requests.get(url).text
    soup = BeautifulSoup(response)

    viaf_key = soup.find('title').text

#     Use VIAF Key to parse VIAF entity.
#     Redirects.
    url = f"https://viaf.org/viaf/{viaf_key}/viaf.xml"
    response = requests.get(url).text

#     Parse XML with namespace from string.
    root = ET.fromstring(response)
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}

#     Gather information.
    titles = root.findall('.//ns:titles', ns)
    
    data['viaf']['works'] = []
    
    for w in titles:
        work = w.findall('./ns:work', ns)
        for t in work:
            title = t.find('./ns:title', ns)
            
            data['viaf']['works'].append(title.text)

CPU times: user 15 µs, sys: 24 µs, total: 39 µs
Wall time: 19.1 µs


## Test

In [5]:
%%time

# locID for Lydia Child
locID = "n80001490"

# snacID
snacID = '84910652'

data = {}

# Get LOC & SNAC data.
data['loc'] = parseLOC(locID)
data['snac'] = parseSNAC(snacID)

# Get Wiki data.
data['wiki'] = locSoup(locID)['wiki']
sparqlWiki(data)

# Get VIAF data.
data['viaf'] = locSoup(locID)['viaf']
parseVIAF(data)

CPU times: user 387 ms, sys: 115 ms, total: 502 ms
Wall time: 6.88 s


## App

In [138]:
%%time

# App configurations
app = JupyterDash(__name__, 
                  external_stylesheets = [dbc.themes.SLATE],
                  meta_tags=[
                      {"name": "viewport", "content": "width=device-width, initial-scale=1"},
                  ],
                 )

# Empty dictionary to store query results.
data = {}

app.config.suppress_callback_exceptions = True

app.layout = html.Div(
    className = 'app-body',
    children = [
        
        html.H1('Info Card'),
        html.P('Description'),
    
        dcc.Dropdown(
            id='entityID',
             options = [
                 {'label': 'Cheverus, Jean-Louis-Aimé- Madeleine Lefebvre de', 'value': 'n92060378'},
                 {'label': 'Child, Lydia Maria', 'value': 'n80001490'}
             ], 
             value = 'n92060378'),
        
        dcc.Checklist(
            id = 'selector',
            options=[
                {'label': 'Library of Congress', 'value': 'LOC'},
                {'label': 'SNAC', 'value': 'SNAC'},
                {'label': 'WikiData (requires LoC input)', 'value': 'WIKI'},
                {'label': 'VIAF (requires LoC input)', 'value': 'VIAF'}
            ],
            value=['LOC']),
        
        html.Button('Submit', id='submit-button', n_clicks=0),
        
        html.Div(className = 'data-information', id = 'data-information'),
        
    ]
)

###########################
######### Callbacks #######
###########################

@app.callback(Output('data-information', 'children'), 
              [Input('submit-button', 'n_clicks'), Input('entityID', 'value'), Input('selector', 'value')],
              [State('data-information', 'children')])
def collect_loc(submit, entityID, selector, children):
    
#     Only run if the n_click 'id' is triggered by the revision-button-container.
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
        
    if changed_id != 'submit-button.n_clicks':
        raise PreventUpdate
        
    children = []
    
    for s in selector:
        if s == "LOC":
            data['loc'] = parseLOC(entityID)

            loc_info = []
            for k,v in data['loc'].items():
                loc_info.append(k + ': ' + v)
                loc_info.append(html.Br())
            
            loc_el = html.Div([
                html.H3("Library of Congress"),
                html.P(loc_info)
            ])

    #             Format LOC data for HTML output.
            children.append(loc_el)

        elif s == "SNAC":
            data['snac'] = parseSNAC(snacID)
            
            snac_info = []
            for k,v in data['snac'].items():
                snac_info.append(k + ': ' + v)
                snac_info.append(html.Br())
            
            snac_el = html.Div([
                html.H3("SNAC"),
                html.P(snac_info)
            ])
            
            children.append(snac_el)

        elif s == "WIKI":
            data['wiki'] = locSoup(locID)['wiki']
#             sparqlWiki(data)
            
            wiki_info = []
            for k,v in data['wiki'].items():
                wiki_info.append(k + ': ' + v)
                wiki_info.append(html.Br())
            
            wiki_el = html.Div([
                html.H3("WikiData"),
                html.P(wiki_info)
            ])
            
            children.append(wiki_el)

        elif s == "VIAF":
            data['viaf'] = locSoup(locID)['viaf']
#             parseVIAF(data)
            
            viaf_info = []
            for k,v in data['viaf'].items():
                viaf_info.append(k + ': ' + v)
                viaf_info.append(html.Br())
            
            viaf_el = html.Div([
                html.H3("VIAF"),
                html.P(viaf_info)
            ])
            
            children.append(viaf_el)

        else:
            print( 'alert/message goes here' )
    
    return children


if __name__ == "__main__":
    app.run_server(mode = 'inline', 
                   debug = True) # mode = 'inline' for JupyterDash

CPU times: user 20.7 ms, sys: 3.86 ms, total: 24.6 ms
Wall time: 326 ms
